From geometry optimizations performed outside this notebook, I have the following equilibrium geometries.

| Theory    | Basis Set | Bondlength |  Percent Error | 
| -------- | ------- | -------------- | -------------- |
| CISDTQ  | 6-311++G**   | 0.915801 |  -0.10 \% |
| CCSD(T) | 6-311++G**    | 0.915795 | -0.11 \% |
 | CCSD(T)  | aug-cc-pVTZ | 0.919085 | 0.25 \% |

The experimental geometry is 0.9168 Angstroms.  The CISDTQ/6-311++G** is really close.  I will go with that as my best answer.



In [3]:
import psi4
import numpy as np

# cisdtq/6-311++G** equilibrium geometry
r_eq_ang = 0.915801

# atomic units to wavenumbers
au_to_wn = 219474.63068

# atomic mass units to atomic units of mass
amu_to_au = 1822.89

# angstroms to atomic units of length (Bohr radii)
au_to_ang = 0.52917721067121

# define the Delta r in angstroms
Delta_r_ang = 0.0005

# define Delta r in atomic units
Delta_r_au = Delta_r_ang / au_to_ang

# define the array of displaced bondlengths for psi4 in Angstroms
r_stencil = np.array([r_eq_ang - 2 * Delta_r_ang, 
                      r_eq_ang - Delta_r_ang, 
                      r_eq_ang, 
                      r_eq_ang + Delta_r_ang,
                      r_eq_ang + 2 * Delta_r_ang])

print(r_stencil)

[0.914801 0.915301 0.915801 0.916301 0.916801]


In [4]:
mol_tmpl = """
H
F 1 **R**
symmetry c2v
"""

options_dict = {
    "basis": "6-311++G**",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    "num_frozen_docc" : 1
}
psi4.set_options(options_dict)

cisdtq_on_stencil = []


for r_val in r_stencil:
    mol_str = mol_tmpl.replace("**R**", str(r_val))
    mol = psi4.geometry(mol_str)
    cisdtq_e = psi4.energy('cisdtq')
    cisdtq_on_stencil.append(cisdtq_e)





Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  5, 4, 3
    Auxiliary basis highest AM E, G, H:  6, 5, 4
    Onebody   basis highest AM E, G, H:  6, 5, 4
    Solid Harmonics ordering:            gaussian

*** tstart() called on CHEMYY31VALT
*** at Tue Apr 16 13:11:50 2024

   => Loading Basis Set <=

    Name: 6-311++G**
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry H          line    42 file /Users/jfoley19/Code/psi4/objdir-Release/stage/share/psi4/basis/6-311ppgss.gbs 
    atoms 2 entry F          line   182 file /Users/jfoley19/Code/psi4/objdir-Release/stage/share/psi4/basis/6-311ppgss.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -------------------------------

In [5]:
print(cisdtq_on_stencil)

[-100.28500546474497, -100.2850067482914, -100.28500746037993, -100.28500760313482, -100.28500717867111]


In [6]:
# compute k from points on this stencil - be sure to divide by Delta_r_au **2 in ATOMIC UNITS!
k_ccsd = (cisdtq_on_stencil[3] - 2 * cisdtq_on_stencil[2] + cisdtq_on_stencil[1]) / Delta_r_au ** 2

# compute g from points on this stencil - be sure to divide by 2*Delta_r_au ** 3 in ATOMIC UNITS!
g_ccsd = (cisdtq_on_stencil[4] - 
          2 * cisdtq_on_stencil[3] + 
          2 * cisdtq_on_stencil[1] - 
          cisdtq_on_stencil[0]) / (2 * Delta_r_au ** 3)


We have the relationships
derivatives at $r_{eq}$ analytically:
\begin{align}
k = 2 D_e \beta^2 \\
g = -6 D_e \beta^3 \tag{10} 
\end{align},
where we can solve for $\beta$ as

$$ \beta = -\frac{2g}{6k} $$

and then for $D_e$ as
$$ D_e = \frac{k}{2 \beta^2} $$

From there, we have 
\begin{equation}
\omega = \sqrt{\frac{2D_e \beta^2}{\mu}} \tag{4}
\end{equation}
and
\begin{equation}
\chi_e = \frac{\hbar \omega}{4 D_e}. \tag{5}
\end{equation}


In [17]:
# print k and g
print(F' k from CISDTQ/6-311++G** is       {k_ccsd:8.2e}')
print(F' g from CISDTQ/6-311++G** is       {g_ccsd:8.2e}')

# compute beta and De from k and g
beta = -2 * g_ccsd / (6 * k_ccsd)
De = k_ccsd / (2 * beta ** 2)

# print beta and De
print(F' beta from CISDTQ/6-311++G** is    {beta:8.2f}')
print(F' De from CISDTQ/6-311++G** is      {De:8.2f}')


# compute reduced mass of HF in atomic mass units
mu_amu = (1.00784 * 18.998403) / (1.00784 + 18.998403)

# convert reduced mass to atomic units
mu = mu_amu * amu_to_au

# print out reduced mass in atomic units
print(F' Reduced mass of HF in atomic units is {mu}')

# compute \omega_e in atomic units
omega_au = np.sqrt( 2 * De * beta ** 2 / mu )

# compute \chi_e in atomic units
xe_au = omega_au / (4 * De)

# print \omega_e and \omega_e * \chi_e in atomic units

# convert \omega_e to wavenumbers
omega_wn = omega_au * au_to_wn

# convert \omega_e * \chi_e to wavenumbers
xewe_wn = xe_au * omega_au * au_to_wn

# print \omega_e and \omega_e * \chi_e in wavenumbers


# experimental values from NIST
we_exp = 4138.32
xe_we_exp = 89.88

print(F' \omega_e in atomic units from CISDTQ/6-311++G** is            {omega_au:8.2e}')
print(F' \omega_e * \chi_e in atomic units from CISDTQ/6-311++G** is   {omega_au * xe_au:8.2e}')
print(F' \omega_e in cm^-1 from CISDTQ/6-311++G** is                   {omega_au * au_to_wn:8.2f}')
print(F' \omega_e * \chi_e in cm^-1 from CISDTQ/6-311++G** is          {omega_au * xe_au * au_to_wn:8.2f}')
print(F' \omega_e in cm^-1 from NIST is                                {we_exp:8.2f}')
print(F' \omega_e * \chi_e in cm^-1 from NIST is                       {xe_we_exp:8.2f}')

re_omega_e = ( omega_wn - we_exp) / we_exp * 100
re_omega_e_chi_e = (xewe_wn - xe_we_exp) / xe_we_exp * 100

print(F' % Error of \omega_e is                                        {re_omega_e:8.2f}')
print(F' % Error of \omega_e * \chi_e is                               {re_omega_e_chi_e:8.2f}')


 k from CISDTQ/6-311++G** is       6.38e-01
 g from CISDTQ/6-311++G** is       -2.51e+00
 beta from CISDTQ/6-311++G** is        1.31
 De from CISDTQ/6-311++G** is          0.18
 Reduced mass of HF in atomic units is 1744.6310991830007
 \omega_e in atomic units from CISDTQ/6-311++G** is            1.91e-02
 \omega_e * \chi_e in atomic units from CISDTQ/6-311++G** is   4.94e-04
 \omega_e in cm^-1 from CISDTQ/6-311++G** is                    4196.11
 \omega_e * \chi_e in cm^-1 from CISDTQ/6-311++G** is            108.51
 \omega_e in cm^-1 from NIST is                                 4138.32
 \omega_e * \chi_e in cm^-1 from NIST is                          89.88
 % Error of \omega_e is                                            1.40
 % Error of \omega_e * \chi_e is                                  20.73


In [21]:
def compute_Morse_transition(i, f, we, xe):
    Ei = we * ((i + 1/2) - xe * (i + 1/2) ** 2 )
    Ef = we * ((f + 1/2) - xe * (f + 1/2) ** 2 )
    deltaE = Ef - Ei
    return deltaE

def compute_Harmonic_transition(i, f, we):
    Ei = we * ((i + 1/2))
    Ef = we * ((f + 1/2) )
    deltaE = Ef - Ei
    return deltaE

cisdtq_Morse_fundamental_au = compute_Morse_transition(0, 1, omega_au, xe_au)
cisdtq_Harmonic_fundamental_au = compute_Harmonic_transition(0, 1, omega_au)
experimental_fundamental_wn = compute_Morse_transition(0, 1, we_exp, xe_we_exp / we_exp)

print(F' Morse Fundamental from CISDTQ:    {cisdtq_Morse_fundamental_au * au_to_wn:.2f} cm^-1')
print(F' Harmonic Fundamental from CISDTQ: {cisdtq_Harmonic_fundamental_au * au_to_wn:.2f} cm^-1')
print(F' Fundamental from Expt:            {experimental_fundamental_wn:.2f} cm^-1')
p_error_fundamental = (cisdtq_Morse_fundamental_au * au_to_wn - experimental_fundamental_wn)/experimental_fundamental_wn * 100
print(F' Percent Error                     {p_error_fundamental:8.2f} %')


 Morse Fundamental from CISDTQ:    3979.09 cm^-1
 Harmonic Fundamental from CISDTQ: 4196.11 cm^-1
 Fundamental from Expt:            3958.56 cm^-1
 Percent Error                         0.52 %


In [22]:
cisdtq_Morse_overtone2_au = compute_Morse_transition(0, 3, omega_au, xe_au)
cisdtq_Harmonic_overtone2_au = compute_Harmonic_transition(0, 3, omega_au)
experimental_overtone2_wn = compute_Morse_transition(0, 3, we_exp, xe_we_exp / we_exp)

print(F' Morse 2nd Overtone from CISDTQ:    {cisdtq_Morse_overtone2_au * au_to_wn:.2f} cm^-1')
print(F' Harmonic 2nd Overtone from CISDTQ: {cisdtq_Harmonic_overtone2_au * au_to_wn:.2f} cm^-1')
print(F' 2nd Overtone from Expt:            {experimental_overtone2_wn:.2f} cm^-1')

p_error_overtone = (cisdtq_Morse_overtone2_au * au_to_wn - experimental_overtone2_wn)/experimental_overtone2_wn * 100
print(F' Percent Error                     {p_error_overtone:8.2f} %')

 Morse 2nd Overtone from CISDTQ:    11286.23 cm^-1
 Harmonic 2nd Overtone from CISDTQ: 12588.34 cm^-1
 2nd Overtone from Expt:            11336.40 cm^-1
 Percent Error                        -0.44 %
